# Reassignment by Section

Here's a look at the initial assignment of reports, and the flow of reports between sections:

In [1]:
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## Getting our Data

We want to organize the data based on sections. Specifically, we want to see:
- The total **count** of reassignments
- The section the report was assigned to **before**
- The section the report is assigned to **after**

We'll expliticly exclude cases where **before** is the same as **after**, because those don't quite make sense to graph.We'll expliticly exclude cases where **before** is the same as **after**, because those don't quite make sense for the trends we're looking for.

Since the actstream table stores reassignments in a string ("Updated from BEFORE to AFTER"), we can do that with a quick bit of SQL.

In [2]:
%%sql

DROP TABLE IF EXISTS reassignments;
CREATE TEMP TABLE reassignments AS 
SELECT
    substring(a.description from '%Updated from "#"___#""%' for '#') AS before,
    substring(a.description from '%Updated from "___" to "#"___#""%' for '#') AS after,
    COUNT(1) count
FROM actstream_action a
WHERE verb='Assigned section:'
GROUP BY before, after;

SELECT * FROM reassignments WHERE before != after ORDER BY before;

before,after,count
ADM,DRS,49
ADM,FCS,1
ADM,VOT,4
ADM,APP,3
ADM,CRM,2
ADM,HCE,3
ADM,EOS,1
ADM,ELS,1
ADM,POL,1
ADM,SPL,3


## Preparing for the Graph

Next, let's run some python on the data to get it ready for graphing.

We'll need to get ready:
- Some info to show for each of the sections on the outside of the ring (a label, count info, etc).
- The connections between the sections, organized in a way that the graph can use it.
- A title for the graph

First, let's bring in some libraries and list the sections and grab the data from our table above:

In [3]:
import pandas
import holoviews

SECTIONS = ['ADM', 'APP', 'CRM', 'DRS', 'ELS', 'EOS', 'FCS', 'HCE', 'IER', 'POL', 'SPL', 'VOT']

reassignments_sql = %sql SELECT * FROM reassignments WHERE before != after ORDER BY before;
reassignments = reassignments_sql.DataFrame().fillna(0)

### Section Info

First, let's define what our sections are. We'll give each one:
- A name
- A numerical **index** (which we'll just make the order it appears in the list above)
- Info to be displayed when you mouse over the section, namely:
  - How many total reassignments that section made
  - How many assignments it sent to each section

In [4]:
section_info = [
    {
        'name': this_section,
        'index': index,
        'Total Reassignments': reassignments[reassignments['before'] == this_section]['count'].sum(),
        **{
            f'Sent to {after}:': reassignments[(reassignments['before'] == this_section) & (reassignments['after'] == after)]['count'].sum()
            for after in SECTIONS
        }
    }
    for index, this_section
    in enumerate(SECTIONS)
]

nodes = holoviews.Dataset(pandas.DataFrame(section_info).fillna(0), 'index')

### Connections

Let's tell the graph what each of the links should be. This needs:
- A **source** section (the position of the **before** section in the list above) 
- A **target** section (the position of the **after** section in the list above)
- The **value** of that link (the number of reports referred from the source to the target section)

In [5]:
reassignments['source'] = reassignments['before'].apply(lambda b: SECTIONS.index(b))
reassignments['index'] = reassignments['source']
reassignments['target'] = reassignments['after'].apply(lambda a: SECTIONS.index(a))
reassignments['value'] = reassignments['count']

links = reassignments[['source', 'target', 'value']]

### Title

Finally, let's add a title for the graph that includes the total number of reassignments.

In [6]:
count_of_all = reassignments['value'].sum()

title = f"Reassignments between sections ({count_of_all} total)"

## Graphing!

Now that we have our nodes, links, and title, we can go ahead and plug that into the graph.

We'll also add some options for colors (based on the section the report was in **before** the reassignment):

In [ ]:
holoviews.extension('bokeh')
holoviews.output(size=300)

chord = holoviews.Chord((links, nodes))
chord.opts(
    holoviews.opts.Chord(
        title=title,
        labels='name',
        cmap='Category20', edge_cmap='Category20',
        edge_color=holoviews.dim('source').str(), node_color=holoviews.dim('index').str(),
))